In [1]:
import dask.dataframe as dd

In [2]:

# Carregar os dados de vendas com calendário
sales_calendar = dd.read_parquet('../data/sales_calendar/')

# Carregar os dados de preços
item_prices = dd.read_parquet('../data/prices/')

In [3]:
# Converter yearweek para string nos dois DataFrames
sales_calendar['yearweek'] = sales_calendar['yearweek'].astype(str)
item_prices['yearweek'] = item_prices['yearweek'].astype(str)

In [4]:
# Fazer o merge dos DataFrames usando 'item', 'store_code', e 'yearweek' como chave
merged_data = sales_calendar.merge(item_prices, on=['item', 'store_code', 'yearweek'], how='left')

# Verificar as primeiras linhas do DataFrame resultante
print(merged_data.head())

                          id                 item     category_x  \
0    SUPERMARKET_3_613_PHI_1    SUPERMARKET_3_613    SUPERMARKET   
1    SUPERMARKET_3_816_PHI_1    SUPERMARKET_3_816    SUPERMARKET   
2     ACCESORIES_1_174_PHI_2     ACCESORIES_1_174     ACCESORIES   
3     ACCESORIES_1_253_PHI_2     ACCESORIES_1_253     ACCESORIES   
4  HOME_&_GARDEN_1_026_PHI_2  HOME_&_GARDEN_1_026  HOME_&_GARDEN   

        department            store store_code        region       d  sales  \
0    SUPERMARKET_3  Midtown_Village      PHI_1  Philadelphia  d_1588      3   
1    SUPERMARKET_3  Midtown_Village      PHI_1  Philadelphia  d_1588      0   
2     ACCESORIES_1         Yorktown      PHI_2  Philadelphia  d_1588      0   
3     ACCESORIES_1         Yorktown      PHI_2  Philadelphia  d_1588      1   
4  HOME_&_GARDEN_1         Yorktown      PHI_2  Philadelphia  d_1588      3   

  yearweek       date event     category_y  sell_price  
0   201523 2015-06-04  <NA>    SUPERMARKET      3.4560  
1 

In [5]:
merged_data.to_parquet('../data/ds_market/', write_index=False)

In [6]:
merged_data.isnull().sum().compute()

id                   0
item                 0
category_x           0
department           0
store                0
store_code           0
region               0
d                    0
sales                0
yearweek             0
date                 0
event         57561115
category_y    12513532
sell_price    12513532
dtype: int64

In [ ]:
# Verificar se há valores ausentes nas colunas principais
missing_values = merged_data.isnull().sum().compute()
print("Valores ausentes em cada coluna:\n", missing_values)


In [ ]:
# Verificar duplicatas com base em 'item', 'store_code', e 'yearweek'
duplicates = merged_data.duplicated(subset=['item', 'store_code', 'yearweek']).compute()
print(f"Número de duplicatas: {duplicates.sum()}")


In [ ]:
# Descrever os dados para verificar os intervalos de valores
print(merged_data[['sell_price', 'sales']].describe().compute())

# Verificar possíveis anomalias como vendas elevadas com preços muito baixos
anomalies = merged_data[(merged_data['sell_price'] < 1) & (merged_data['sales'] > 100)].compute()
print("Possíveis anomalias:\n", anomalies)


In [ ]:
# Verificar se o yearweek está ordenado corretamente
sorted_weeks = merged_data[['yearweek', 'date']].drop_duplicates().compute().sort_values('date')
print("Verificação de yearweek ordenado por data:\n", sorted_weeks.head())

# Verificar a associação de eventos com vendas em semanas importantes (e.g., eventos como 'Black Friday')
events_with_sales = merged_data[merged_data['event'].notnull() & (merged_data['sales'] > 0)].compute()
print("Vendas durante eventos importantes:\n", events_with_sales[['event', 'yearweek', 'sales']])


In [ ]:
# Calcular estatísticas descritivas para identificar possíveis outliers
sales_summary = merged_data['sales'].describe().compute()
price_summary = merged_data['sell_price'].describe().compute()

print("Resumo das vendas:\n", sales_summary)
print("Resumo dos preços de venda:\n", price_summary)

# Verificar outliers em vendas e preços
outliers_sales = merged_data[merged_data['sales'] > sales_summary['75%'] + 1.5 * (sales_summary['75%'] - sales_summary['25%'])].compute()
outliers_price = merged_data[merged_data['sell_price'] > price_summary['75%'] + 1.5 * (price_summary['75%'] - price_summary['25%'])].compute()

print("Outliers em vendas:\n", outliers_sales)
print("Outliers em preços de venda:\n", outliers_price)


In [ ]:
# Calcular correlação entre vendas e preço de venda
correlation = merged_data[['sell_price', 'sales']].corr().compute()
print("Correlação entre preço de venda e vendas:\n", correlation)


In [ ]:
# Verificar se as vendas aumentam durante eventos específicos
event_sales = merged_data[merged_data['event'].notnull()].groupby('event')['sales'].sum().compute()
print("Vendas durante eventos:\n", event_sales)
